In [1]:
import pandas as pd
import awswrangler as wr
from boto3 import Session

In [2]:
session = Session(profile_name='globant')
con = wr.mysql.connect("globant_connection", boto3_session=session)
schema = 'globant'

In [3]:
def upload_data(df: pd.DataFrame, schema: str, table_name:str):
    df_with_nan = df[df.isnull().any(axis=1)]
    count_error_records = df_with_nan.shape[0]
    df = df.dropna()
    count_success_records = df.shape[0]

    wr.mysql.to_sql(
        df=df,
        table=table_name,
        schema=schema,
        con=con,
        mode="overwrite"
    )

    cursor = con.cursor()
    sql = f"ALTER TABLE {schema}.{table_name} MODIFY id INT PRIMARY KEY"
    cursor.execute(sql)
    con.commit()

    return {
        'records_error': count_error_records,
        'records_success': count_success_records,
    }

In [4]:
table_name = 'departments'
file_path = f'./{table_name}.csv'

df = pd.read_csv(file_path, header=None)
df.columns = ['id', 'department']

upload_data(
    df=df,
    schema='globant',
    table_name=table_name
)

{'records_error': 0, 'records_success': 12}

In [5]:
table_name = 'jobs'
file_path = f'./{table_name}.csv'

# Assuming the first row contains headers
df = pd.read_csv(file_path, header=None)

# If you want to explicitly set column names, you can do it after reading the CSV
df.columns = ['id', 'job']

upload_data(
    df=df,
    schema='globant',
    table_name=table_name
)

{'records_error': 0, 'records_success': 183}

In [6]:
table_name = 'hired_employees'
file_path = f'./{table_name}.csv'

df = pd.read_csv(file_path, header=None)
df.columns = ['id', 'name', 'datetime', 'department_id', 'job_id']

df['department_id'] = df['department_id'].astype('Int64')
df['job_id'] = df['job_id'].astype('Int64')
df['job_id'] = df['job_id'].astype('Int64')
df['datetime'] = pd.to_datetime(df['datetime'])

upload_data(
    df=df,
    schema='globant',
    table_name=table_name
)

{'records_error': 70, 'records_success': 1929}